In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 100)

arquivo1 = 'data/apy.csv'
arquivo2 = 'data/API_IND_DS2_en_csv_v2_200361.csv'

data1 = pd.read_csv(arquivo1)              # Dados sobre a produção agricola na India
data2 = pd.read_csv(arquivo2,skiprows=4)   # Dados do world bank da India


# Após ler os datasets, manipular os dados e  gerar o dataset final

In [ ]:
data1.shape

In [ ]:
data1.columns

In [ ]:
data1.columns = ['State_Name', 'District_Name', 'Year', 'Season', 'Crop', 'Area',
       'Production']

In [ ]:
data1.columns

In [ ]:
data2.columns

In [ ]:
# Serão usados apenas algumas colunas do dataset original
data2= data2[['Indicator Name','1997','1998','1999','2000','2001',
              '2002','2003','2004','2005','2006','2007','2008','2009',
              '2010','2011','2012','2013','2014','2015']]

In [ ]:
# Apenas o indicator GDP será selecionado
data2_GDP = data2[data2['Indicator Name']== 'GDP (current US$)']

In [ ]:
# Linhas são transformadas em colunas
data2_GDP_tr= data2_GDP.transpose()

In [ ]:
data2_GDP_tr.to_csv('data/data2_GDP_tr.csv') 

In [ ]:
# Removendo o index
temp = 'data/data2_GDP_tr.csv'
data2 = pd.read_csv(temp,skiprows=1)

In [ ]:
# Renomeando as colunas de data2
data2.columns = ['Year','GDP (current US$)']

In [ ]:
data2.head()

In [ ]:
data1.columns

In [ ]:
data2.columns

In [ ]:
# 
df = pd.merge(data2, data1, on=["Year"])


In [ ]:
df.head()

In [ ]:
# Dataset final
#df.to_csv('data/india_crop_gdp_1997_2015.csv') 

# Algoritmo 1:  
# Objetivo: descobrir a CROP (Cultivo). 
# A partir de 3 dados aleatórios (AREA, STATE, DISTRICT) inseridos pelo usuário, seja calculado o resultado de CROP

In [ ]:
df = pd.read_csv("data/india_crop_gdp_1997_2015.csv")

In [ ]:
df.head()

In [ ]:
# Serão usados apenas algumas colunas do dataset original
df1 = df [['Area','State_Name','District_Name','Crop']]

In [ ]:
df1.head()

In [ ]:
# Outliers em Area
#sns.boxplot(x=df1['Area'])

In [ ]:
# Z score indica o quanto os dados estã afastados da média dos dados

z = np.abs(stats.zscore(df1.Area))

# Dados acima de 3 z scores são considerados outliers
df1_o = df1[(z < 2.5)]


In [ ]:
# Tamanho original do dataset
df1.shape

In [ ]:
# Tamanho depois de retirados os outliers
df1_o.shape

In [ ]:
#sns.boxplot(x=df1_o['Area'])

In [ ]:
df1 = df1_o

In [ ]:
# Frequencia de distribuição das classes 
Crop_classe = pd.crosstab(index=df1["Crop"],columns="count")  
Crop_classe.columns = ["Frequencia"]
Crop_classe.head()

# São mais de 120 classes e estão desbalanceadas no dataset

In [ ]:
#df1['Crop'].value_counts().plot(kind='bar', figsize=(6,6))
#plt.title('Classe Crop')
#plt.xlabel('Classe')
#plt.ylabel('Frequencia')
#plt.show()

In [ ]:
# df11 recebe todos atributos de df1
df11 = df1 
df1.info()

In [ ]:
# df1 remove a variável classe Crop
label_df = df1['Crop']
df1.drop('Crop', axis = 1, inplace=True)
df1.head()

In [ ]:
label_df

In [ ]:
# Atributos strings mudam para numérico

le = LabelEncoder()

df1['State_Name'] = le.fit_transform(df1['State_Name']) # State_Name
df1['District_Name'] = le.fit_transform(df1['District_Name']) # District_Name
label_df = le.fit_transform(label_df) # Crop


In [ ]:
# OnehotEncoder ira transformar os valores em arrays para evitar influencia de valores altos
onehotencoder = OneHotEncoder(categorical_features = [1,2])
df1 = onehotencoder.fit_transform(df1).toarray()

In [ ]:

pl_random_forest = Pipeline(steps=[('random_forest', RandomForestClassifier())])

scores = cross_val_score(pl_random_forest, df1, label_df, cv=10,scoring='accuracy')

print('Accuracy for RandomForest : ', scores.mean())
# Accuracy for RandomForest :  0.2568389205158886


In [ ]:
pl_xgb = Pipeline(steps=

                  [('xgboost', xgb.XGBClassifier(objective='multi:softmax'))])

scores = cross_val_score(pl_xgb, df1, label_df, cv=10)

print('Accuracy for XGBoost Classifier : ', scores.mean())

In [ ]:
# Atributos previsores e classe
previsores = df11.iloc[:, 0:3].values
classe = df11.iloc[:, 3].values



In [ ]:
# Aplicando a mesma escala nos dados
# Normalizando
previsores = MinMaxScaler().fit_transform(previsores)

# Padronizando os dados (0 para a média, 1 para o desvio padrão)
previsores = StandardScaler().fit_transform(previsores)


# Divisão em treino e teste
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.3, random_state=0)


In [ ]:
# Criar o modelo, treinar e avaliar


#classificador = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=123)
#classificador = DecisionTreeClassifier(criterion='entropy', random_state=123)
classificador= KNeighborsClassifier(n_neighbors = 5)

# Treinamento
classificador.fit(previsores_treinamento, classe_treinamento)

# Previsao
previsoes = classificador.predict(previsores_teste)

# Avaliar a acurácia
acurácia = accuracy_score(classe_teste, previsoes)

print('Acurácia:', acurácia)
# Acurácia: 0.2667527898411302


In [ ]:
# Previsoes feitas com o modelo gerado
#le.inverse_transform(previsoes)

# Algoritmo 2: 
# Objetivo: descobrir a PRODUCTION (Produção agrícola de um cultivo). A partir de 3 dados aleatórios (AREA, CROP e GDP) inseridos pelo usuário, seja calculado o resultado de PRODUCTION

In [ ]:
df = pd.read_csv("data/india_crop_gdp_1997_2015.csv")

In [ ]:
# Dados disponíveis
df.head(3)

In [ ]:
# Na coluna Production há um sinal de '=' em 3000 linhas, 1% do total. Essas linhas serão removidas do dataset.

df.loc[df.Production == '=', 'Production'] = np.nan
df = df.dropna()
df.Production = df.Production.astype(float)

In [ ]:
# De string para numérico
df.Crop = le.fit_transform(df.Crop) 

In [ ]:
# Serão usados apenas algumas colunas do dataset original
df['GDP'] = df.iloc[:,1] # Renomeando a coluna
df1 = df [['Area','Crop','GDP','Production']]

In [ ]:
# Detectando Outliers em Area
#sns.boxplot(x=df1['Area'])

In [ ]:
# Z score indica o quanto os dados estã afastados da média dos dados

z = np.abs(stats.zscore(df1.Area))


# Dados acima de 3 z scores são considerados outliers
df1_o = df1[(z < 2.5)]

In [ ]:
# Z score indica o quanto os dados estã afastados da média dos dados

z = np.abs(stats.zscore(df1_o.GDP))

# Dados acima de 3 z scores são considerados outliers
df1_o = df1_o[(z < 2.5)]

In [ ]:
# Z score indica o quanto os dados estã afastados da média dos dados

z = np.abs(stats.zscore(df1_o.Production))

# Dados acima de 3 z scores são considerados outliers
df1_o = df1_o[(z < 2.5)]

In [ ]:
df = df1_o

In [ ]:
df

In [ ]:
# Criar o modelo e treinar

param = {
    'objective': 'reg:linear', 
    "booster" : "gbtree",
    'eta': 0.03,
    'max_depth':10,
    'subsample':0.9,
    'colsample_bytree':0.7,
    #'eval_metric': 'mae',
    'silent' : 1  
}

features = ['Area', 'Crop', 'GDP']

X_train, X_test, y_train, y_test = train_test_split(df[features], np.log1p(df['Production']), 
                                                    test_size = 0.3, random_state = 42)

dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_test, y_test)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]





In [ ]:
gbm = xgb.train(
            param, 
            dtrain, 
            1000,
            evals=watchlist,
            early_stopping_rounds=100, 
            verbose_eval=100
)

In [ ]:
predicted = gbm.predict(xgb.DMatrix(X_test))

In [ ]:
# Resultado - Quanto menor melhor a previsão
mse = mean_squared_error(y_test, predicted)
print("MSE do modelo é: ", mse)

In [ ]:
# Resultado - Coeficiente de Determinação, quanto mais próximo de 1 melhor
r2 = r2_score(y_test, predicted)
print("R2 do modelo é: ", r2)

In [ ]:
print('Valor previsto:', np.expm1(predicted[2]))

In [ ]:
print('Valor de teste:', np.expm1(y_test.values[2]))